In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from collections import Counter
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import seaborn as sns

In [ ]:
import pyLDAvis
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis.gensim_models as gensimvis
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample
from pprint import pprint
from gensim.models import CoherenceModel

In [ ]:
dataset = pd.read_csv("D:/Stella/Documents/10_Academy/Twitter-Data-Analysis/data/covid191.csv")


In [ ]:
tweets_text=dataset[['clean_text']]
tweets_text.head(2)

In [ ]:
words = [[text for text in doc.split()] for doc in tweets_text['clean_text']]
#words

In [ ]:
# Create Dictionary
covid_dict = corpora.Dictionary(words)

# Create Corpus
texts = words

# Term Document Frequency
covidcorpus = [covid_dict.doc2bow(text) for text in texts]

# View
#print(corpus[:1][0][:10])

In [ ]:
num_topics=10
# Build LDA model
lda_model = models.LdaMulticore(corpus=covidcorpus,
                                       id2word=covid_dict,
                                        alpha=0.01,
                                        num_topics=num_topics, 
                                        random_state=100)

# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[covidcorpus]

In [ ]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(covidcorpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=words, dictionary=covid_dict, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)

In [ ]:
#"""Save model objects"""
SaveLoad.save(lda_model,'covidLDAmodel')
corpora.MmCorpus.serialize('covidCorpus.mm', covidcorpus)
covid_dict.save('covidDictionary.dict')

In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook()
ldaViz = gensimvis.prepare(lda_model, covidcorpus, covid_dict)

In [ ]:
ldaViz

In [2]:
ldaViz

C:\Users\Stella\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
C:\Users\Stella\Anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps
C:\Users\Stella\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
dataset = pd.read_csv("D:/Stella/Documents/10_Academy/Twitter-Data-Analysis/data/covid191.csv")


In [4]:
tweets_text=dataset[['clean_text']]
tweets_text.head(2)

,clean_text
0,"['africa', '""in', 'midst', 'full-blown', 'thir..."
1,"['dr', 'moeti', 'head', 'africa', 'one', 'best..."


In [5]:
words = [[text for text in doc.split()] for doc in tweets_text['clean_text']]
#words

In [6]:
# Create Dictionary
covid_dict = corpora.Dictionary(words)

# Create Corpus
texts = words

# Term Document Frequency
covidcorpus = [covid_dict.doc2bow(text) for text in texts]

# View
#print(corpus[:1][0][:10])

In [15]:
num_topics=10
# Build LDA model
lda_model = models.LdaMulticore(corpus=covidcorpus,
                                       id2word=covid_dict,
                                        alpha=0.01,
                                        num_topics=num_topics, 
                                        random_state=100)

# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[covidcorpus]

[(0,
  '0.050*"\'vaccine\'," + 0.022*"\'india\'," + 0.018*"\'#covid\'," + '
  '0.013*"\'amp\'," + 0.011*"\'flu\'," + 0.011*"[\'get\'," + '
  '0.011*"\'shingle\'," + 0.011*"\'including\'," + 0.011*"\'visit\'," + '
  '0.011*"\'wait\']"'),
 (1,
  '0.067*"\'vaccine\'," + 0.056*"\'another\'," + 0.054*"\'expired\'," + '
  '0.052*"\'israel\']" + 0.052*"\'tried\'," + 0.052*"[\'liar\'," + '
  '0.052*"\'war\'," + 0.052*"\'crime\'," + 0.052*"\'load\'," + '
  '0.010*"\'#covid\',"'),
 (2,
  '0.046*"\'vaccine\'," + 0.034*"\'dose\'," + 0.030*"[\'pin\'," + '
  '0.030*"\'min\'," + 0.030*"\'code[]\'," + 0.030*"\'limit\'," + '
  '0.030*"\'[]\'," + 0.030*"\'age\'," + 0.023*"\'capacity\']" + '
  '0.022*"\'covishield\',"'),
 (3,
  '0.027*"\'vaccine\'," + 0.020*"\'#covid\'," + 0.017*"\'third\'," + '
  '0.017*"\'midst\'," + 0.016*"\'case\'," + 0.015*"\'india\'," + '
  '0.014*"[\'africa\'," + 0.012*"\'wave"\'," + 0.012*"\'full-blown\'," + '
  '0.012*"\'head\',"'),
 (4,
  '0.039*"\'vaccine\'," + 0.016*"\'#covid

In [16]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(covidcorpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=words, dictionary=covid_dict, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -6.390430051260688

 Ldamodel Coherence Score/Accuracy on Tweets:  0.31817256021368556


In [17]:
#"""Save model objects"""
SaveLoad.save(lda_model,'covidLDAmodel')
corpora.MmCorpus.serialize('covidCorpus.mm', covidcorpus)
covid_dict.save('covidDictionary.dict')

In [18]:
import pyLDAvis
pyLDAvis.enable_notebook()
ldaViz = gensimvis.prepare(lda_model, covidcorpus, covid_dict)

In [19]:
ldaViz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.110516  0.037529       1        1  18.058901
1      0.199506 -0.153438       2        1  12.853171
2     -0.155280 -0.089718       3        1  11.533683
6      0.062898  0.048020       4        1  10.571101
5      0.012850  0.048304       5        1  10.281977
7     -0.120519 -0.066715       6        1   8.125300
3      0.026484  0.039600       7        1   8.010695
0      0.005198  0.166345       8        1   7.741211
8     -0.126709  0.013504       9        1   6.741984
4     -0.014944 -0.043430      10        1   6.081978, topic_info=          Term        Freq       Total Category  logprob  loglift
956   'tried',  707.000000  707.000000  Default  30.0000  30.0000
958   ['liar',  704.000000  704.000000  Default  29.0000  29.0000
955    'load',  704.000000  704.000000  Default  28.0000  28.0000
953   'crime',  705.000000  705.000000  Default  27.0000  27.0000
957     'war',  708.000000  708.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
11   'africa',   34.502484  582.355565  Topic10  -4.8015  -0.0262
9     'wave"',   24.755796  226.943189  Topic10  -5.1335   0.5842
431    ['pin',   26.975655  457.153841  Topic10  -5.0476  -0.0302
427   'limit',   26.755486  456.811543  Topic10  -5.0558  -0.0377
421     'age',   26.545281  469.563793  Topic10  -5.0637  -0.0731

[754 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
468       1  0.101029    '"can',
468       2  0.707202    '"can',
468      10  0.101029    '"can',
3788      2  0.632199  '"great',
3788      5  0.210733  '"great',
...     ...       ...        ...
2239      5  0.161170  ['zweli',
2239      7  0.017908  ['zweli',
2239      8  0.340247  ['zweli',
2239      9  0.017908  ['zweli',
2239     10  0.017908  ['zweli',

[2327 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 3, 7, 6, 8, 4, 1, 9, 5])